## Mapping Feature

In [1]:
import pandas as pd

df = pd.DataFrame([
    ['green', 'M', 10.1, 'class1'],
    ['red', 'L', 13.5, 'class2'],
    ['blue', 'XL', 15.3, 'class1']
])

df.columns = ['color', 'size', 'price', 'classlabel']

df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


In [2]:
size_mapping = {
    'XL': 3,
    'L': 2,
    'M': 1
}

df['size'] = df['size'].map(size_mapping)
df

# 将整数型数值转换回原来字符串的形式
# 定义反向映射字典
# inv_size_mapping = {
#     v: k for k, v in size_mapping.items()
# }

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


## Label Coding

In [3]:
import numpy as np

class_mapping = {label: idx for idx, label in enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

In [4]:
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


In [5]:
# 反向映射键值
# 将转换后的分类标记匹配到原来的字符串
# inv_class_mapping = {v: k for k, v in class_mapping.items()}
# df['classlabel'] = df['classlabel'].map(inv_class_mapping)
# df

In [6]:
# 调用sklearn中的LabelEncoder类实现
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

array([0, 1, 0], dtype=int64)

In [7]:
# 利用inverse_transform方法将分类的整数型标签转换回原来的字符串形式
y = class_le.inverse_transform(y)
y

array([0, 1, 0], dtype=int64)

## HotEncoding for Name Features

In [8]:
X = df[['color', 'size', 'price']].values
color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:, 0])
X
# 此时blue=0， green=1， red=2
# 在机器学习算法里，此时会假设blue<green<red
# 这是处理分类数据时最常见的错误

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

为了避免上述出现的错误，这里使用的常见方案为**独热编码**

blue样本编码为：blue=1, green=0, red=0。

In [19]:
from sklearn.preprocessing import OneHotEncoder

X = df[['color']].values
ohe = OneHotEncoder()
ohe.fit_transform(X).toarray()

array([['green'],
       ['red'],
       ['blue']], dtype=object)

In [22]:
pd.get_dummies(df[['price', 'color', 'size']])

,price,size,color_blue,color_green,color_red
0,10.1,1,0,1,0
1,13.5,2,0,0,1
2,15.3,3,1,0,0


In [23]:
pd.get_dummies(df[['price', 'color', 'size']], drop_first=True)
# drop_first=True 删除第一列

,price,size,color_green,color_red
0,10.1,1,1,0
1,13.5,2,0,1
2,15.3,3,0,0


但这种独热编码还会带来多重共线性，对某些方法这个可能是个问题（尤其是需要矩阵求逆的方法）

解决办法是可以直接从独热编码矩阵中删除一个特征列，并且不会失去重要的信息

例：如果删除了color_blue，如果color_red=0 color_green=0，这意味着剩下的观察结果必然是蓝色